In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from collections import OrderedDict

### Importando tabelas

In [105]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('data_science/cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso
id_cet

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151
20152
20161
20162
20171
20172


17    10320810
Name: id_curso, dtype: int64

In [132]:
df_turmas = pd.read_csv('data_science/turmas-2010-2017-ok.csv')
df_turmasFilt = df_turmas[(df_turmas['ano'] >= 2014) & (df_turmas['nivel_ensino'] == 'GRADUAÇÃO')]
# df_turmasFilt = df_turmas[df_turmas['nivel_ensino'] == 'GRADUAÇÃO']
df_turmasFilt.ano.value_counts()

/home/andreza/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2014    22531
2015    22467
2017    22309
2016    21946
Name: ano, dtype: int64

In [133]:
df_componentes = pd.read_csv('data_science/componentes-curriculares-presenciais-ok.csv')

In [134]:
print(df_turmasFilt.columns)
print(df_componentes.columns)

Index(['Unnamed: 0', 'id_turma', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes'],
      dtype='object')
Index(['Unnamed: 0', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_pro

### Merges

In [135]:
df_turmasNome = pd.merge(df_turmasFilt, df_componentes, left_on='id_componente_curricular', right_on='id_componente')
df_turmasNome.columns

Index(['Unnamed: 0_x', 'id_turma', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes',
       'Unnamed: 0_y', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_programa', 'modalidade

In [136]:
disciplinas = ['CÁLCULO I','CÁLCULO II','CÁLCULO III','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO','PROBABILIDADE E ESTATÍSTICA',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','QUÍMICA GERAL','ALGEBRA LINEAR','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS']

In [137]:
df_auxTurmas = df_turmasNome[df_turmasNome['nome'].isin(disciplinas)]

In [138]:
colunas = ['id_turma', 'codigo', 'nome', 'pre_requisito']
df_auxTurmas = df_auxTurmas[colunas]

df_auxAll = pd.merge(df_matriculas, df_auxTurmas, on='id_turma')

colunas2 = ['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome']
df_auxAll = df_auxAll[colunas2].drop_duplicates().dropna()

In [139]:
df_auxAll.head()

,descricao,discente,id_turma,media_final,codigo,nome
2,REPROVADO,b70cfc26a1b6eb885a529e60c07354cd,1226773,3.4,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
16,REPROVADO,970cbdd2d4d5d75cc06acf545766eba1,1226773,0.0,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
22,REPROVADO,7ba56f4d760d44531b3f8220c58e947b,1226773,0.0,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
52,APROVADO,dfdf0394ccc40abbcbc8aa6ec6e49969,1220717,5.4,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
58,REPROVADO,db1fd7f22172df2305735834721d934a,1220717,4.7,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS


In [140]:
df_auxAll.nome.value_counts()

CÁLCULO I                             6912
VETORES E GEOMETRIA ANALÍTICA         5779
PRÉ-CÁLCULO                           4276
PROBABILIDADE E ESTATÍSTICA           4129
CÁLCULO II                            3928
QUÍMICA GERAL                         3874
LINGUAGEM DE PROGRAMAÇÃO              3859
COMPUTAÇÃO NUMÉRICA                   3229
CÁLCULO III                           3216
MECÂNICA DOS SÓLIDOS                  2969
CIÊNCIA E TECNOLOGIA DOS MATERIAIS    2962
INTRODUÇÃO À FÍSICA CLÁSSICA I        2918
LÓGICA DE PROGRAMAÇÃO                 2762
INTRODUÇÃO À FÍSICA CLÁSSICA II       2512
MECÂNICA DOS FLUIDOS                  2483
INTRODUÇÃO À FÍSICA CLÁSSICA III      1882
ALGEBRA LINEAR                         514
Name: nome, dtype: int64

### Correlação

In [141]:
# apenas os aprovados
df_auxAprovados = df_auxAll[df_auxAll['descricao'].str.contains('APROVADO')]
df_auxAprovados.columns

Index(['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome'], dtype='object')

In [142]:
df_auxAprovados.descricao.value_counts()

APROVADO POR NOTA    18193
APROVADO              9587
Name: descricao, dtype: int64

In [143]:
discentes= df_auxAprovados.discente.unique()

In [144]:
isso = df_auxAll[(df_auxAll['nome']=='CÁLCULO I') & (df_auxAll['discente']=='fe16b848a14a942f60497ffaf3bde2f9')].media_final
isso

818    5.8
Name: media_final, dtype: float64

In [145]:
df_auxAprovados.columns

Index(['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome'], dtype='object')

In [146]:
df_calc1 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO I']
df_calc1 = df_calc1[['discente','media_final']]
print(df_calc1.shape)

df_calc2 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO II']
df_calc2 = df_calc2[['discente','media_final']]
print(df_calc2.shape)

df_calc3 = df_auxAprovados[df_auxAprovados['nome']=='CÁLCULO III']
df_calc3 = df_calc3[['discente','media_final']]
print(df_calc3.shape)

df_vga = df_auxAprovados[df_auxAprovados['nome']=='VETORES E GEOMETRIA ANALÍTICA']
df_vga = df_vga[['discente','media_final']]
print(df_vga.shape)

df_pc = df_auxAprovados[df_auxAprovados['nome']=='PRÉ-CÁLCULO']
df_pc = df_pc[['discente','media_final']]
print(df_pc.shape)

df_pe = df_auxAprovados[df_auxAprovados['nome']=='PROBABILIDADE E ESTATÍSTICA']
df_pe = df_pe[['discente','media_final']]
print(df_pe.shape)

df_f1 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA I']
df_f1 = df_f1[['discente','media_final']]
print(df_f1.shape)

df_f2 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA II']
df_f2 = df_f2[['discente','media_final']]
print(df_f2.shape)

df_f3 = df_auxAprovados[df_auxAprovados['nome']=='INTRODUÇÃO À FÍSICA CLÁSSICA III']
df_f3 = df_f3[['discente','media_final']]
print(df_f3.shape)

df_lop = df_auxAprovados[df_auxAprovados['nome']=='LÓGICA DE PROGRAMAÇÃO']
df_lop = df_lop[['discente','media_final']]
print(df_lop.shape)

df_lp = df_auxAprovados[df_auxAprovados['nome']=='LINGUAGEM DE PROGRAMAÇÃO']
df_lp = df_lp[['discente','media_final']]
print(df_lp.shape)

df_cn = df_auxAprovados[df_auxAprovados['nome']=='COMPUTAÇÃO NUMÉRICA']
df_cn = df_cn[['discente','media_final']]
print(df_cn.shape)

df_qg = df_auxAprovados[df_auxAprovados['nome']=='QUÍMICA GERAL']
df_qg = df_qg[['discente','media_final']]
print(df_qg.shape)

df_al = df_auxAprovados[df_auxAprovados['nome']=='ALGEBRA LINEAR']
df_al = df_al[['discente','media_final']]
print(df_al.shape)

df_ms = df_auxAprovados[df_auxAprovados['nome']=='MECÂNICA DOS SÓLIDOS']
df_ms = df_ms[['discente','media_final']]
print(df_ms.shape)

df_mf = df_auxAprovados[df_auxAprovados['nome']=='MECÂNICA DOS FLUIDOS']
df_mf = df_mf[['discente','media_final']]
print(df_mf.shape)

df_ctm = df_auxAprovados[df_auxAprovados['nome']=='CIÊNCIA E TECNOLOGIA DOS MATERIAIS']
df_ctm = df_ctm[['discente','media_final']]
print(df_ctm.shape)

(2425, 2)
(1809, 2)
(1448, 2)
(2216, 2)
(2047, 2)
(2016, 2)
(1315, 2)
(1479, 2)
(1171, 2)
(1468, 2)
(1799, 2)
(1494, 2)
(2107, 2)
(272, 2)
(1449, 2)
(1516, 2)
(1749, 2)


In [147]:
df_auxAprovados[df_auxAprovados['nome']=='ALGEBRA LINEAR'].codigo.value_counts()

MAT0343    252
MAT0369     20
Name: codigo, dtype: int64

In [148]:
teste = pd.DataFrame()
teste['discente'] = discentes

teste = pd.merge(teste, df_al, on='discente')
teste = teste.rename(columns={"media_final": 'ALGEBRA LINEAR'})
teste.head()

,discente,ALGEBRA LINEAR
0,5efce18e36d1989aba6c0614f334fcc5,5.7
1,607b89c7663813b0577a1ebf2315f80c,6.3
2,485126e1f950797fa5d32f01a91792fc,5.5
3,4dc29bb23c754d71d6e311a55d333729,9.2
4,ba2010fa68da2c25c85bc64c3a48e620,7.2


In [149]:
df_correr = pd.DataFrame()
df_correr['discente'] = discentes

disciplinas = ['ALGEBRA LINEAR','CÁLCULO I','CÁLCULO II','CÁLCULO III','PROBABILIDADE E ESTATÍSTICA',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'QUÍMICA GERAL']

dataFrames = [df_al,df_calc1,df_calc2,df_calc3,df_pe,df_lp,df_cn,df_ms,df_mf,df_ctm,df_vga,df_pc,df_f1,df_f2,df_f3,df_lop,df_qg]

# dataFrames,disciplinas
for i in range(0, 16):
    print(i)
    df_correr = pd.merge(df_correr, dataFrames[i], on='discente')
    df_correr = df_correr.rename(columns={"media_final": disciplinas[i]})
    display(df_correr.head())
    
df_correr.head()

0


,discente,ALGEBRA LINEAR
0,5efce18e36d1989aba6c0614f334fcc5,5.7
1,607b89c7663813b0577a1ebf2315f80c,6.3
2,485126e1f950797fa5d32f01a91792fc,5.5
3,4dc29bb23c754d71d6e311a55d333729,9.2
4,ba2010fa68da2c25c85bc64c3a48e620,7.2


1


,discente,ALGEBRA LINEAR,CÁLCULO I
0,5d5d904110ee38b37643a03d54e2f2b3,8.2,5.4
1,a78f2930ba6869e2757b6bac7740dd7c,6.2,5.8
2,6e30e4a4d0cb70fcaf0a4cd5f547e33c,7.3,5.2
3,ba0544830d336471d753e673ffd825ff,5.6,6.9
4,7001bc2dcedafd568676d83007d2eed9,6.0,6.1


2


...

14


,ALGEBRA LINEAR,CÁLCULO I,CÁLCULO II,CÁLCULO III,PROBABILIDADE E ESTATÍSTICA,LINGUAGEM DE PROGRAMAÇÃO,COMPUTAÇÃO NUMÉRICA,MECÂNICA DOS SÓLIDOS,MECÂNICA DOS FLUIDOS,CIÊNCIA E TECNOLOGIA DOS MATERIAIS,VETORES E GEOMETRIA ANALÍTICA,PRÉ-CÁLCULO,INTRODUÇÃO À FÍSICA CLÁSSICA I,INTRODUÇÃO À FÍSICA CLÁSSICA II,discente,INTRODUÇÃO À FÍSICA CLÁSSICA III


15


,ALGEBRA LINEAR,CÁLCULO I,CÁLCULO II,CÁLCULO III,PROBABILIDADE E ESTATÍSTICA,LINGUAGEM DE PROGRAMAÇÃO,COMPUTAÇÃO NUMÉRICA,MECÂNICA DOS SÓLIDOS,MECÂNICA DOS FLUIDOS,CIÊNCIA E TECNOLOGIA DOS MATERIAIS,VETORES E GEOMETRIA ANALÍTICA,PRÉ-CÁLCULO,INTRODUÇÃO À FÍSICA CLÁSSICA I,INTRODUÇÃO À FÍSICA CLÁSSICA II,INTRODUÇÃO À FÍSICA CLÁSSICA III,discente,LÓGICA DE PROGRAMAÇÃO


,ALGEBRA LINEAR,CÁLCULO I,CÁLCULO II,CÁLCULO III,PROBABILIDADE E ESTATÍSTICA,LINGUAGEM DE PROGRAMAÇÃO,COMPUTAÇÃO NUMÉRICA,MECÂNICA DOS SÓLIDOS,MECÂNICA DOS FLUIDOS,CIÊNCIA E TECNOLOGIA DOS MATERIAIS,VETORES E GEOMETRIA ANALÍTICA,PRÉ-CÁLCULO,INTRODUÇÃO À FÍSICA CLÁSSICA I,INTRODUÇÃO À FÍSICA CLÁSSICA II,INTRODUÇÃO À FÍSICA CLÁSSICA III,discente,LÓGICA DE PROGRAMAÇÃO
